In [2]:
from pathlib import Path

cfg_path = Path.home().resolve() / "config"

if cfg_path.is_file():
    print(f"Found config: {cfg_path}")
else: 
    cfg_dir = input("Enter directory with config: ")
    cfg_path = Path(cfg_dir).resolve() / "config"
    if cfg_path.is_file():
        print(f"Found config: {cfg_path}")
    else:
        print("ERROR: Re-run this cell and give correct directory")

Found config: /home/user1-selab3/Documents/research-shradha/kube/SPT/config


In [3]:
from shutil import copy
from pathlib import Path

dest_dir = Path.home().resolve() / ".kube"
dest_dir.mkdir(exist_ok=True)
dest_path = dest_dir / "config"

if dest_path.is_file():
    print("SUCCESS: Config correctly configured")
elif cfg_path.is_file() and dest_dir.is_dir():
    copy(cfg_path, dest_path)
    print("SUCCESS: Copied config")
else:
    print("ERROR: Ensure you have correct config path")

SUCCESS: Config correctly configured


In [4]:
! kubectl config view --minify -o jsonpath='{..namespace}'

gp-engine-unoselab01

In [5]:
from jinja2 import Template

# read in the template
with open('/home/user1-selab3/Documents/research-shradha/kube/kube2/example1-kube/CODE-RajulShakywar/CODE/example/yaml/yaml_templates/pvc_template.yml') as file_:
    template = Template(file_.read())

In [6]:
PVC_NAME = 'pvc-shradha-sptcode-gp-engine-unoselab01'

pvc_spec = template.render(name=PVC_NAME)

In [7]:
print(pvc_spec)

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: pvc-shradha-sptcode-gp-engine-unoselab01
spec:
  storageClassName: rook-cephfs-central
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 50Gi


In [8]:
with open("/home/user1-selab3/Documents/research-shradha/kube/SPT/kube-pvc-spt.yml", "w") as file:
    file.write(pvc_spec)

In [9]:
! kubectl create -f ../SPT/kube-pvc-spt.yml

persistentvolumeclaim/pvc-shradha-sptcode-gp-engine-unoselab01 created


In [6]:
! kubectl create -f ../SPT/kube-pod-spt.yml

pod/shradha-sptcode-gp-engine-unoselab01-pod created


In [7]:
! kubectl get pvc

NAME                                       STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS          AGE
pvc-shradha-sptcode-gp-engine-unoselab01   Bound    pvc-a0bcf939-01d0-470c-99d5-97c4420642b4   50Gi       RWX            rook-cephfs-central   14h


In [110]:
! kubectl get pods

NAME                                       READY   STATUS    RESTARTS   AGE
shradha-sptcode-gp-engine-unoselab01-pod   1/1     Running   0          3h27m


In [111]:
! kubectl cp /home/user1-selab3/Documents/research-shradha/kube/SPT/run_install.sh shradha-sptcode-gp-engine-unoselab01-pod:/data/run_install.sh

In [112]:
! kubectl cp /home/user1-selab3/Documents/research-shradha/deploy-spt-code shradha-sptcode-gp-engine-unoselab01-pod:/data/deploy-spt-code

In [113]:
! kubectl exec -it shradha-sptcode-gp-engine-unoselab01-pod -- ls -l /data/deploy-spt-code


total 1
drwxr-xr-x 3 root root   1 Aug 15 23:23 asts
drwxr-xr-x 4 root root  17 Aug 15 23:25 data_shradha
drwxr-xr-x 6 root root   4 Aug 15 23:27 dataset
drwxr-xr-x 6 root root   5 Aug 16 17:37 deploy-spt-code
drwxr-xr-x 4 root root   9 Aug 15 23:27 spt-code
-rw-rw-r-- 1 1002 1002 365 Aug 15 23:27 steps.txt


In [114]:
! kubectl exec shradha-sptcode-gp-engine-unoselab01-pod -- cat /data/deploy-spt-code/spt-code/sources/main.py

import torch

import argparse
import random
import numpy as np
import logging
import os
import sys
import time
from prettytable import PrettyTable

from args import add_args
from train import train
from pre_train import pre_train


def main(args):

    model = None
    vocab = None
    if args.do_pre_train:
        model, vocab = pre_train(args=args)

    if args.do_fine_tune or args.only_test:
        train(args=args,
              trained_model=model,
              trained_vocab=vocab)


if __name__ == '__main__':

    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.register('type', 'bool', lambda v: v.lower() in ['yes', 'true', 't', '1', 'y'])

    add_args(parser)

    main_args = parser.parse_args()

    # define and make dirs
    # Root directory for the output of this run
    main_args.output_root = os.path.join(
        '..',
        'outputs',
        '{}_{}'.format(main_args.model_name, time.strftime('%Y%m%d_%H%M%S', time.lo

In [115]:
! kubectl exec shradha-sptcode-gp-engine-unoselab01-pod -- cat /data/run_install.sh

#!/bin/bash

#!/bin/bash
set -e
cd /data

echo "Activating environment..."
eval "$(conda shell.bash hook)"
conda activate env_mlm

echo "Installing dependencies..."
pip install psutil gputil;
pip3 install numpy pandas matplotlib;
pip3 install ipykernel;
pip3 install ipywidgets;
pip3 install tqdm;
pip3 install torch torchvision torchaudio;
pip install transformers;
pip install scikit-learn;
pip install prettytable;
pip install dataclasses;
pip install chardet;
pip install rouge==1.0.0;
pip install accelerate;
pip install typing;
pip install antlr4-tools;
conda install -c conda-forge tensorboard;
pip install tree_sitter==0.19.0;
pip install antlr4-python3-runtime==4.9.2;
pip install nltk;
echo '--------------------------------------------------------'
echo 'Installation completed'
echo '--------------------------------------------------------'

cd /data/deploy-spt-code/spt-code/sources;
python main.py --do-pre-train --pre-train-tasks cap --batch-size 16 --eval-batch-size 32 --cuda-visibl

In [119]:
! kubectl cp /home/user1-selab3/Documents/research-shradha/deploy-spt-code/spt-code/sources/data/asts/ast_parser.py shradha-sptcode-gp-engine-unoselab01-pod:/data/deploy-spt-code/spt-code/sources/data/asts/ast_parser.py

In [23]:
# ! kubectl cp /home/user1-selab3/Documents/research-shradha/deploy-spt-code/spt-code/sources/data/asts/build_lib.py shradha-sptcode-gp-engine-unoselab01-pod:/data/deploy-spt-code/spt-code/sources/data/asts/build_lib.py

In [131]:
! kubectl cp /home/user1-selab3/Documents/research-shradha/deploy-spt-code/spt-code/sources/data/data_utils.py shradha-sptcode-gp-engine-unoselab01-pod:/data/deploy-spt-code/spt-code/sources/data/data_utils.py

In [6]:
! kubectl exec shradha-llmast-gp-engine-unoselab01-pod -- cat /data/deploy-spt-code/spt-code/sources/args.py

import dataclasses
from dataclasses import dataclass, field
import os
import enums


@dataclass
class RuntimeArguments:
    """Arguments for runtime."""

    do_pre_train: bool = field(
        default=False,
        metadata={'action': 'store_true',
                  'help': 'Whether to pre-train'}
    )

    pre_train_tasks: str = field(
        default=','.join(enums.PRE_TRAIN_TASKS),
        metadata={'help': 'Pre-training tasks in order, split by commas, '
                          'for example (default) {}'.format(','.join(enums.PRE_TRAIN_TASKS))}
    )

    do_fine_tune: bool = field(
        default=True,
        metadata={'action': 'store_true',
                  'help': 'Whether to fine_tune, task can be specific by `--task`'}
    )

    only_test: bool = field(
        default=False,
        metadata={'action': 'store_true',
                  'help': 'Whether to test only'}
    )

    task: str = field(
        default='search',
        metadata={'help': 'Downstream task',
 

In [120]:
! kubectl exec shradha-sptcode-gp-engine-unoselab01-pod -- cat /data/deploy-spt-code/spt-code/sources/data/asts/ast_parser.py


import tree_sitter
from tree_sitter import Language, Parser
import re

import enums


LANGUAGE = {
            # enums.LANG_GO: Language('data/asts/build/my-languages.so', 'go'),
            # enums.LANG_JAVASCRIPT: Language('data/asts/build/my-languages.so', 'javascript'),
            # enums.LANG_PYTHON: Language('data/asts/build/my-languages.so', 'python'),
            enums.LANG_JAVA: Language('data/asts/build/my-languages.so', 'java')#,
            # enums.LANG_PHP: Language('data/asts/build/my-languages.so', 'php'),
            # enums.LANG_RUBY: Language('data/asts/build/my-languages.so', 'ruby'),
            # enums.LANG_C_SHARP: Language('data/asts/build/my-languages.so', 'c_sharp')
            }

# LANGUAGE = {enums.LANG_GO: Language('build/my-languages.so', 'go'),
#             enums.LANG_JAVASCRIPT: Language('build/my-languages.so', 'javascript'),
#             enums.LANG_PYTHON: Language('build/my-languages.so', 'python'),
#             enums.LANG_JAVA: Language('build/my

In [121]:
! kubectl exec -it shradha-sptcode-gp-engine-unoselab01-pod -- ls -R /data/


/data/:
deploy-spt-code  output-spt-code-2024-.log  run_install.sh

/data/deploy-spt-code:
asts  data_shradha  dataset  deploy-spt-code  spt-code	steps.txt

/data/deploy-spt-code/asts:
pretrain

/data/deploy-spt-code/asts/pretrain:
ast_jdt_output_file.txt

/data/deploy-spt-code/data_shradha:
NL_methods_test.txt	    raw_methods_valid.txt
NL_methods_train.txt	    source_methods_test.txt
NL_methods_valid.txt	    source_methods_train.txt
gen			    source_methods_valid.txt
raw_methods_asts_test.txt   target_methods_test.txt
raw_methods_asts_train.txt  target_methods_train.txt
raw_methods_asts_valid.txt  target_methods_valid.txt
raw_methods_test.txt	    tokenized_javakw
raw_methods_train.txt

/data/deploy-spt-code/data_shradha/gen:
raw_methods_test_gen.txt      source_methods_valid_gen.txt
raw_methods_train_gen.txt     target_methods_test_gen.txt
raw_methods_valid_gen.txt     target_methods_train_gen.txt
source_methods_test_gen.txt   target_methods_valid_gen.txt
source_methods_train_gen.txt


In [3]:
! kubectl delete pod shradha-sptcode-gp-engine-unoselab01-pod  

pod "shradha-sptcode-gp-engine-unoselab01-pod" deleted


In [4]:
! kubectl delete job job01-sptcode-shradha-gp-engine-unoselab01

job.batch "job01-sptcode-shradha-gp-engine-unoselab01" deleted


In [134]:
! kubectl create -f ../SPT/kube-job-spt.yml

job.batch/job01-sptcode-shradha-gp-engine-unoselab01 created


In [1]:
! kubectl get pods

NAME                                               READY   STATUS      RESTARTS   AGE
job01-sptcode-shradha-gp-engine-unoselab01-kqjqp   0/1     Completed   0          74m
shradha-sptcode-gp-engine-unoselab01-pod           1/1     Running     0          4h58m


In [136]:
!kubectl get jobs

NAME                                         COMPLETIONS   DURATION   AGE
job01-sptcode-shradha-gp-engine-unoselab01   0/1           10s        10s


In [137]:
! kubectl logs -f job01-sptcode-shradha-gp-engine-unoselab01-kqjqp

Activating environment...
Installing dependencies...
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=43a8a443b615576375ca30a557eac5f12e73429c1719cffbcc775fc61e70288c
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built gputil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 772.9 kB/s eta 0:00:00
   ━━━━━

In [83]:
!kubectl get pods --selector=job-name=job01-sptcode-shradha-gp-engine-unoselab01

NAME                                               READY   STATUS      RESTARTS   AGE
job01-sptcode-shradha-gp-engine-unoselab01-qcxf4   0/1     Completed   0          3m9s


In [37]:
!kubectl logs -f job01-sptcode-shradha-gp-engine-unoselab01-9whv9 > log.txt

In [38]:
! kubectl get job job01-sptcode-shradha-gp-engine-unoselab01

NAME                                         COMPLETIONS   DURATION   AGE
job01-sptcode-shradha-gp-engine-unoselab01   0/1           7m43s      7m43s


In [39]:
! kubectl logs job/job01-sptcode-shradha-gp-engine-unoselab01

Found 2 pods, using pod/job01-sptcode-shradha-gp-engine-unoselab01-9whv9
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=5e3d87e14153504fb04ba1c4164b7ff447375596a3a6cace7e976843eaef6707
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built gputil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 131.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 139.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 129.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 1.5 MB/

In [12]:
! kubectl get pods --selector=job-name=job01-sptcode-shradha-gp-engine-unoselab01

NAME                                               READY   STATUS              RESTARTS   AGE
job01-sptcode-shradha-gp-engine-unoselab01-hlt89   0/1     ContainerCreating   0          43s


In [73]:
! kubectl logs job01-sptcode-shradha-gp-engine-unoselab01-l4zhb

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=63197e3af63d36d0066717c2013a4e069904e4ecda5f33267ff8de5cc88b7788
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built gputil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 139.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.4 M

In [43]:
!kubectl get pods --selector=job-name=job01-sptcode-shradha-gp-engine-unoselab01

NAME                                               READY   STATUS   RESTARTS   AGE
job01-sptcode-shradha-gp-engine-unoselab01-4kk9r   0/1     Error    0          9m13s
job01-sptcode-shradha-gp-engine-unoselab01-9whv9   0/1     Error    0          10m


In [46]:
! kubectl exec -it job01-sptcode-shradha-gp-engine-unoselab01-9whv9  -- /bin/bash


error: cannot exec into a container in a completed pod; current phase is Failed


In [48]:
! kubectl cp job01-sptcode-shradha-gp-engine-unoselab01-9whv9:/data/output-spt-code-2024-<DATE>.log ./output-spt-code-2024-<DATE>.log


/bin/bash: DATE: No such file or directory
